Note: It is important to specify the versions of libraries used after getting the code to run first time. These libraries are updated quite often and this usually results in error and incompatibilty between liberaries.

Use:
pip freeze > requirements.txt
to generate a file containing all the details about libraries used and their versions. This will come handy in case of replicating code.

On kaggle keep the environment pinned to original to avoid conflict in future.

In [2]:
!pip install -q langchain==0.1.6
!pip install -q chromadb

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 23.8.0 requires cubinlinker, which is not installed.
cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 23.8.0 requires ptxcompiler, which is not installed.
cuml 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
keras-cv 0.8.2 requires keras-core, which is not installed.
keras-nlp 0.8.1 requires keras-core, which is not installed.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
cudf 23.8.0 requires cuda-python<12.0a0,>=11.7.1, but you have cuda-python 12.3.0 which is incompatible.
cudf 23.8.0 requires pandas<1.6.0dev0,>=1.3, but you have pandas 2.1.4 which is incompatible.
cudf 23.8.0 requires protobuf<5,>=4.21, but you have protobuf 3.20.3 which is incompatible.
cuml 23.8.0 r

In [3]:
!pip install -q torch bitsandbytes transformers==4.21.3 accelerate

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
chromadb 0.4.24 requires tokenizers>=0.13.2, but you have tokenizers 0.12.1 which is incompatible.
kaggle-environments 1.14.3 requires transformers>=4.33.1, but you have transformers 4.21.3 which is incompatible.
llama-index-embeddings-huggingface 0.1.4 requires transformers<5.0.0,>=4.37.0, but you have transformers 4.21.3 which is incompatible.


In [4]:
!pip install -q pypdf
!pip install -q sentence-transformers

In [7]:
#loading documents from folder containing the pdfs
from langchain.document_loaders import PyPDFDirectoryLoader
loader = PyPDFDirectoryLoader("/kaggle/input/diabetes-documents")
docs = loader.load()

In [8]:
#splitting text to make embeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000)
docs = text_splitter.split_documents(docs)

In [9]:
#defining embedding function
#sentence transformer model used to save memory
from langchain.embeddings import HuggingFaceEmbeddings

model_kwargs = {'device':'cuda'}
encode_kwargs = {'normalize_embeddings':False}
embeddings_func = HuggingFaceEmbeddings(
  model_name = 'sentence-transformers/all-mpnet-base-v2',  
  model_kwargs = model_kwargs,
  encode_kwargs=encode_kwargs
)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [11]:
#Create a vector store for embeddings in local storage using chromaDB
from langchain.vectorstores import Chroma

#for creating the vector store we pass chunks of docs and embedding function
#These two lines are used to create the store and used only once or when docs are changed
#Comment out these lines if vector store already created
db=Chroma.from_documents(docs, embeddings_func, persist_directory="chroma_db")
db.persist()

#The following line is used to load Vector store already created
# db = Chroma(persist_directory="/kaggle/working/chroma_db", embedding_function=embeddings_func)

In [12]:
#This code contains standard loading procedure for a LLM using hugging face libraries. 
#llama-2 7b-chat is used for both tokenization and generation
#To access llama-2 in kaggle go to +Add input -> Models -> llama-2
#Copy it's path here as base_model_name

import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    pipeline
)
from langchain import HuggingFacePipeline

base_model_name = "/kaggle/input/llama-2/pytorch/7b-chat-hf/1"

tokenizer = AutoTokenizer.from_pretrained(base_model_name, max_length=512, padding=True, truncation=True)


base_model = AutoModelForCausalLM.from_pretrained(base_model_name)
base_model.config.use_cache = False

# pipeline is create to use the LLM to generate responses
# It integrates tokenization and generation for us

pipe = pipeline("text2text-generation", model=base_model, tokenizer=tokenizer)
llm = HuggingFacePipeline(pipeline = pipe)

2024-03-21 11:22:29.836045: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-21 11:22:29.836169: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-21 11:22:29.970173: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/configuratio

In [13]:
#print information about pipeline
# print(llm)

In [14]:
#define a template for augmented prompt sent to the LLM
#The {context} and {question} are added by Retrieval Model itself

from langchain.prompts import PromptTemplate

template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Keep the answer as concise as possible. 
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

In [ ]:
# RetrievalQA by langchain lets us augment the prompt before passing it to HuggingFacePipeline 

from langchain.chains import RetrievalQA 

qa_chain = RetrievalQA.from_chain_type(
    llm, retriever=db.as_retriever(), chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)

question = "How can language barriers be addressed in diabetes care?"
result = qa_chain({"query": question})
print(result["result"])

/opt/conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1197: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
